In [1]:
%pip install fastapi uvicorn scipy librosa numpy




[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Function for processing data chunks
# 1) Normalize between -1 and 1
# 2) Bandpass 50-5000 Hz 
# 3) Extract MFCCs (40 of them)

from scipy import signal
import librosa
import numpy as np

RATE = 22050
sos = signal.butter(5, [50, 5000], 'bandpass', fs=RATE, output='sos')

def get_mfccs(audio): 
   
    try:
        audio = 2*((audio-min(audio))/(max(audio)-min(audio)))-1

        #Filter
        audio = signal.sosfilt(sos, audio) 

        mfccs = librosa.feature.mfcc(y=audio, sr=RATE, n_mfcc=40)

    except Exception as e:
            print("Error extracting features")
            return None
     
    return mfccs

In [3]:
from fastapi import FastAPI, File, UploadFile
from scipy import signal
import librosa
import numpy as np
import io
import soundfile as sf
import nest_asyncio
import uvicorn
from fastapi.responses import JSONResponse
import asyncio

# Initialize nest_asyncio to allow running FastAPI within a Jupyter notebook
nest_asyncio.apply()

# Create the FastAPI app
app = FastAPI()

# Define the endpoint to process the audio file
@app.post("/process-audio/")
async def process_audio(file: UploadFile = File(...)):
    # Read the uploaded file
    audio_data, sample_rate = sf.read(io.BytesIO(await file.read()), dtype='float32')

    # Your processing code here (using scipy and librosa)
    # For example, let's compute the short-time Fourier transform (STFT)
    stft = librosa.stft(audio_data)
    
    # Convert the result to a more convenient format for demonstration purposes
    result = np.abs(stft).tolist()

    return JSONResponse(content={"stft": result})

# Run the app
def run_app():
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)

    loop = asyncio.get_event_loop()
    loop.run_until_complete(server.serve())

import threading
thread = threading.Thread(target=run_app)
thread.start()


# run_app()


INFO:     Started server process [31674]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [4]:
import http.server
import socketserver
import threading

PORT = 8001

Handler = http.server.SimpleHTTPRequestHandler

httpd = socketserver.TCPServer(("", PORT), Handler)

print(f"Serving at port {PORT}")

threading.Thread(target=httpd.serve_forever).start()


Serving at port 8001


127.0.0.1 - - [07/Jun/2024 12:49:37] "GET /index.html HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2024 12:49:38] code 404, message File not found
127.0.0.1 - - [07/Jun/2024 12:49:38] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Jun/2024 12:49:48] "GET /index.html HTTP/1.1" 304 -
